In [86]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [51]:
df = pd.read_csv('df.csv', encoding='windows-1252')

In [55]:
df.columns = df.columns.str.strip().str.replace(" ", "_")

In [ ]:
#Removendo colunas que não ajudam na previsao
df.drop(columns=['Row_ID', 'Order_ID', 'Customer_ID', 'Customer_Name', 'Product_ID', 'Product_Name'], inplace=True)

In [ ]:
#Conversão de dados categoricos
label_encoders = {}
categorical_cols = ['Ship_Mode', 'Segment', 'Country', 'City', 'State', 'Region', 'Category', 'Sub-Category']

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [ ]:
#Datas para valores numericos
df['Order_Date'] = pd.to_datetime(df['Order_Date']).astype('int64') / 10**9
df['Ship_Date'] = pd.to_datetime(df['Ship_Date']).astype('int64') / 10**9

In [ ]:
#Remoçao dos outliers 
df = df[(df['Profit'] > df['Profit'].quantile(0.01)) & (df['Profit'] < df['Profit'].quantile(0.99))]

In [ ]:
#Variaveis de treino e teste
X = df.drop(columns=['Profit'])
y = df['Profit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [149]:
#Treinamento do modelo
model = GradientBoostingRegressor(n_estimators=1500, learning_rate=0.01, max_depth=6, random_state=42)
model.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=6, n_estimators=1500,
                          random_state=42)

In [150]:
#Realizar as previsoes
y_pred = model.predict(X_test)

In [ ]:
#Visualizaçao do das metricas de avaliaçao do modelo
metrics = {
    "MAE": mean_absolute_error(y_test, y_pred),
    "MSE": mean_squared_error(y_test, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "R²": r2_score(y_test, y_pred)
}

print(metrics)

{'MAE': 11.34427148756334, 'MSE': 766.0583372223672, 'RMSE': 27.67775889089229, 'R²': 0.8594699629243363}


In [ ]:
#Visualizaçao de dados reais e previstos para comparaçao
comparacao = pd.DataFrame({'Real': y_test.values, 'Previsto': y_pred})
print(comparacao.head(10))

       Real    Previsto
0    8.8784   19.061794
1 -135.0870 -124.774964
2  -14.7594  -13.595834
3   12.2850   10.627744
4   17.4720   17.568152
5    5.7510    5.846655
6   41.7060   25.601211
7    4.6344   -0.708848
8   18.6624   18.477137
9   12.8744   21.550802


In [ ]:
# Gera insights com base nos resultados do modelo
import openai

openai.api_key = "SUA_API_KEY_AQUI"

def gerar_insights(metrics):
    prompt = f"""
    Tenho um modelo de Machine Learning que prevê lucro com base em vendas. Aqui estão as métricas de desempenho:

    - MAE (Erro Absoluto Médio): {metrics['MAE']:.2f}
    - MSE (Erro Quadrático Médio): {metrics['MSE']:.2f}
    - RMSE (Raiz do Erro Quadrático Médio): {metrics['RMSE']:.2f}
    - R² (Coeficiente de Determinação): {metrics['R²']:.4f}

    Com base nesses dados, forneça um insight sobre a qualidade do modelo e sugestões para melhorá-lo.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "Você é um especialista em Machine Learning."},
                  {"role": "user", "content": prompt}]
    )

    return response["choices"][0]["message"]["content"]

insight = gerar_insights(metrics)
print(insight)